## Run this first if you want NDVI/NDWI basemaps

In [16]:
import ee
ee.Authenticate(force=True)



Successfully saved authorization token.


## Vibes

In [8]:
%load_ext autoreload
%autoreload 2

from geovibes.ui import GeoVibes

vibes = GeoVibes(
    duckdb_directory = '/Users/christopherren/geovibes/local_databases/',
    start_date = '2024-01-01',
    end_date = '2025-01-01',
    verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing GeoVibes...
✅ Earth Engine initialized with user credentials
  Found DB: /Users/christopherren/geovibes/local_databases/alabama_google_satellite_embeddings_v1_2024_2025_25_0_10_metadata.db with Index: /Users/christopherren/geovibes/local_databases/alabama_google_satellite_embeddings_v1_2024_2025_25_0_10_faiss_4096_64_8.index
  Found DB: /Users/christopherren/geovibes/local_databases/new-mexico_google_satellite_embeddings_v1_2024_2025_25_0_10_metadata.db with Index: /Users/christopherren/geovibes/local_databases/new-mexico_google_satellite_embeddings_v1_2024_2025_25_0_10_faiss_4096_64_8.index
Found 2 database(s) in /Users/christopherren/geovibes/local_databases/
✅ Using geometry file: /Users/christopherren/geovibes/geometries/alabama.geojson
✅ Using geometry file: /Users/christopherren/geovibes/geometries/new-mexico.geojson
💾 Connecting to local database: /Users/christopherren/geovibes/l

Loading Earth Engine basemaps: 100%|██████████| 4/4 [00:24<00:00,  6.05s/it]

✅ Earth Engine basemaps added successfully!
✅ Boundary layer added successfully


In [ ]:
Y


In [ ]:
Y
